In [9]:
from recipe_scrapers import scrape_me
import pandas as pd
from time import sleep
from requests import TooManyRedirects


def scrape_and_print(url, dont_print=True):
    scraper = scrape_me(url)
    
    ingredients = '\n'.join([
        '* {}'.format(ing)
        for ing in scraper.ingredients()
    ])
    
    print_kwargs = {
        'title': scraper.title(),
        'tot_time': scraper.total_time(),
        'ingredients': ingredients,
        'instructions': scraper.instructions(),
    }
    
    if not dont_print:
        print(
            '=== {title} ========================\n\n'
            'Time needed: {tot_time} minutes\n\n'
            'INGREDIENTS: \n{ingredients}\n\n'
            'INSTRUCTIONS:\n{instructions}'
            '====================================='
            '\n\n\n'
            .format(**print_kwargs)
        )
    
    return {
        'title': scraper.title(),
        'tot_time': scraper.total_time(),
        'ingredients': scraper.ingredients(),
        'instructions': scraper.instructions(),
    }

In [10]:
example_urls = [
    'https://www.allrecipes.com/recipe/236700/beef-bourguignon-without-the-burgundy/',
    'https://www.allrecipes.com/recipe/12682/apple-pie-by-grandma-ople/',
    'https://www.allrecipes.com/recipe/240376/homemade-pepperoni-pizza/'
]

for url in example_urls:
    _ = scrape_and_print(url, dont_print=False)

=== Beef Bourguignon Without the Burgundy  ========================

Time needed: 160 minutes

INGREDIENTS: 
* 1 (2 1/2 pound) boneless beef chuck roast, cut into 2-inch cubes
* salt and freshly ground black pepper to taste
* 2 tablespoons vegetable oil
* 1 onion, chopped
* 1 tablespoon butter
* 2 tablespoons flour
* 2 cups Merlot wine
* 2 cups beef broth
* 2 carrots, cut into 1-inch pieces
* 2 stalks celery, cut into 1/2-inch pieces
* 4 sprigs fresh thyme
* 1 bay leaf

INSTRUCTIONS:
Generously season beef with salt and pepper.
Heat oil in a large Dutch oven over high heat. Cook and stir beef cubes in hot oil until browned on all sides, 10 to 15 minutes. Transfer meat to a plate.
Cook and stir onion, butter, and a pinch of salt in the Dutch oven until onion begins to sweat. Stir flour into onion mixture; cook and stir until onion starts to soften, 3 to 4 minutes.
Pour wine into onion mixture; bring to a simmer and cook until wine is reduced by half, about 10 minutes. Return beef and an

In [11]:
DF_recipe_urls = pd.concat([
    pd.read_csv('./data/recipes{}.txt'.format(i), header=None)
    for i in ['', 1, 2, 3]
])

In [12]:
def strip_and_clean(x):
    return x.strip().replace('<loc>', '').replace('</loc>', '')

In [13]:
unique_urls = sorted(map(strip_and_clean, DF_recipe_urls[0].unique()))

In [14]:
counter = 0
scrapes = {}
error_counter = 0

for url in unique_urls:
    if error_counter > 10:
        print('exceeded 10 errors')
        break
        
    try:
        scrape = scrape_and_print(url)
    except TooManyRedirects:
        error_counter += 1
        print('error for: {}\nerror_counter = {}'.format(url, error_counter))
        continue
        
    scrapes[url] = scrape
    counter += 1
    sleep(1)
    if counter % 100 == 0:
        print(counter)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
error for: https://www.allrecipes.com/recipe/13511/baked-ziti-with-turkey-meatballs/
error_counter = 1
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
168

ConnectionError: ('Connection aborted.', OSError(101, 'Network is unreachable'))

In [18]:
new_unique_urls = [url for url in unique_urls if url not in scrapes]

In [19]:
for url in new_unique_urls:
    if error_counter > 10:
        print('exceeded 10 errors')
        break
        
    try:
        scrape = scrape_and_print(url)
    except TooManyRedirects:
        error_counter += 1
        print('error for: {}\nerror_counter = {}'.format(url, error_counter))
        continue
        
    scrapes[url] = scrape
    counter += 1
    sleep(1)
    if counter % 100 == 0:
        print(counter)

error for: https://www.allrecipes.com/recipe/13511/baked-ziti-with-turkey-meatballs/
error_counter = 3
50600
50700
50800
50900
51000
51100
51200
51300
51400
51500
51600
51700
51800
51900
52000
52100
52200
52300
52400
52500
52600
52700
52800
52900
53000
53100
53200
53300
53400
53500
53600
53700
53800
53900
54000
54100
54200
54300
54400
54500
54600
54700
54800
54900
55000
55100
55200
55300
55400
55500
55600
55700
55800
55900
56000
56100
56200
56300
56400
56500
56600
56700
56800
56900
57000
57100
57200
57300
57400
57500
57600
57700
57800
57900
58000
58100
58200
58300
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800


In [20]:
import pickle as pkl


with open('allscrapes.pkl', 'wb') as out:
    pkl.dump(scrapes, out)

In [21]:
with open('allscrapes.pkl', 'rb') as out:
    unpickled = pkl.load(out)